%%latex
\tableofcontents

%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

# Introduction

This project deals with the problem of object tracking in a static-background environment. The goal is to find and track objects in a video sequence. The methods used are based on the work of Nummiaro et al. \cite{nummiaro2002}, where applying colour distributions, make the model more robust to brief and partial occlusion, rotation, and scale changes, especially efficient and useful in the case of single-target tracking.
The videos used in this project are from the MOT17 challenge dataset \cite{mot17}, available [here](https://motchallenge.net/data/MOT17/).


In [1]:
# general
from src.utils import *
import os
import time
# computation and vision
import cv2
import numpy as np
import pandas as pd
# plotting
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# names = ['F_PG1_Subject_2_L', 'F_CP_1_Subject_1']
names = ['MOT17-09-DPM-raw']
# extensions = {'F_PG1_Subject_2_L': '.avi', 'F_CP_1_Subject_1': '.mp4'}
extensions = {'MOT17-09-DPM-raw': '.mp4'}
videos = {}
for name in names:
    videos[name] = load_video('./MOT/'+name+extensions[name], show=False, f=0.3)


# Particle-Filter Tracking

This method, based on the work *An Adaptive Colour-Based Particle Filter* \cite{nummiaro2002}, and follows a top-down approach, where it generates object hypotheses and attempts to verify them with the data. Using the colour distributions, this method only attempts to match only objects that have a similar histogram and build on top of particle filtering.

## Definition
Particle filtering tracks the state of an object described by a vector $X_t$ while an observation vector $Z_t$ keeps track of all observations up to time $t$. These filters are often used for non-Gaussian posterior density $p(X_t|Z_t)$ and observation density $p(z_t|X_t)$. The key idea is approximating the probability distribution by a weighted samples set $S$ where each sample denotes a hypothetical state of the object with a discrete sampling probability distribution $\pi$.

The sample set evolves by propagating each sample according to a model, then each element is weighted in terms of observation and $N$ samples are drawn by choosing a particular sample with probabily $\pi^{(n)} = p(z_t|X_t = s_t^{(n)}).$ The mean state of an object is estimated at each time $t$ by:
$$ E[S] = \sum_{n=1}^{N}{\pi^{(n)}s^{(n)}}$$
By modelling uncertainty, the model keeps the options open and can consider multiple hypotheses and choose the closest. By keeping the less likely states in memory briefly, the model can deal with short-term occlusion.

### Colour Distribution
The colour distribution are used to provide robustness to non-rigidity, rotation, scaling, and partial occlusion. By discretizing the distributions into $m$ bins, histograms produced are produced in HSV colour space to allow for less sensitivity to illumination changes (fewer bins in the V channel). The distribution is determined inside an upright elliptic region with half axes $H_x$ and $H_y$. Smaller weights are assigned to pixels farther from the centre of the region using a weighting function.
$$k(r) = \left \{ \begin{array}{ccl}1-r^2 & r < 1 \\ 0 & otherwise \end{array}\right.$$
where $r$ is the distance from the centre of the region.
The colour distribution $p_y = {p_y^{(u)}}_{u=1..m}$ is defined as:
$$p_y^{(u)} = f \sum_{i=1}^{I}{k(\frac{||y-x_i||}{a})\delta[h(x_i) - u]}$$
where $f$ is the normalization factor, $I$ is the number of pixels in region, $\delta$ is the Kronecker delta function, and $a=\sqrt{H_x^2+H_y^2}$.
Considering the colour histograms $p$ and $q$, the similarity measure is defined using Bhattacharyya coefficient:
$$\rho[p,q] = \sum_{u=1}^{m}{\sqrt{p^{(u)}q^{(u)}}}$$
and hence the Bhattacharyya distance, which updates the a priori distribution given by particle filter, is:
$$d[p,q] = \sqrt{1-\rho[p,q]}$$


Each sample in the distribution represents an ellipse given by $state = {x, y, \dot{x}, \dot{y}, H_x, H_y, \dot{a}}$, where $(x,y)$ is the coordinate of the central point, $(H_x, H_y)$ are the half axes, $(\dot{x}, \dot{y})$ is the motion, and $\dot{a}$ is the scale change.
The sample set is then propagated using:
$$s_t = A s_{t-1} + w_{t-1}$$
where $A$ defines the deterministic component of the model and $w_{t-1}$ is a multivariate Gaussian random variable.

## Algorithm and Implementation
The algorithm has four main steps following model initialization:
1. **Select**: selects $N$ samples from set $S_{t-1}$ with probability $\pi_{t-1}^{(n)}$.
2. **Propagate**: propagates each sample from set $S'_{t-1}$ by the equation given above.
3. **Observe**: observes the colour distributions using the method described.
4. **Estimate**:

In [11]:
# reload utils
from importlib import reload
import src.utils
import src.ParticleFilter 
reload(src.utils)
reload(src.ParticleFilter)
from src.utils import *
from src.ParticleFilter import ParticleFilter

In [29]:
name = 'MOT17-09-DPM-raw'
sframe = 50
eframe = 150
p_count = 30
get_points(videos[name][sframe])

66 57
84 59
56 128
94 123


In [33]:
0.5 * (94 - 60)

17.0

In [21]:
frames = np.copy(videos[name][sframe:eframe])
PF = ParticleFilter(frames, particle_count=p_count, init_x=77, init_y=93, init_Hx=17, init_Hy=35, out_path='./output/')

In [23]:
# pf_df = pd.DataFrame(columns=['Test', 'Time', 'Particle Count'])
# or import from file
pf_df = pd.read_csv('./output/pf_df.csv')

In [24]:
start = time.time()
while PF.f_idx < len(PF.frames) - 1:
    PF.select()
    PF.propagate()
    PF.observe()
    PF.estimate()
pf_time = time.time() - start

selection on frame 1
x: 251, y: 89, Hx: 10, Hy: 24
selection on frame 2
x: 251, y: 90, Hx: 10, Hy: 24
selection on frame 3
x: 251, y: 90, Hx: 9, Hy: 24
selection on frame 4
x: 250, y: 90, Hx: 10, Hy: 24
selection on frame 5
x: 249, y: 91, Hx: 10, Hy: 24
selection on frame 6
x: 249, y: 90, Hx: 10, Hy: 24
selection on frame 7
x: 248, y: 91, Hx: 10, Hy: 24
selection on frame 8
x: 248, y: 90, Hx: 10, Hy: 23
selection on frame 9
x: 247, y: 90, Hx: 10, Hy: 24
selection on frame 10
x: 246, y: 90, Hx: 10, Hy: 24
selection on frame 11
x: 246, y: 89, Hx: 10, Hy: 24
selection on frame 12
x: 244, y: 88, Hx: 10, Hy: 24
selection on frame 13
x: 245, y: 89, Hx: 9, Hy: 23
selection on frame 14
x: 245, y: 90, Hx: 10, Hy: 24
selection on frame 15
x: 244, y: 89, Hx: 9, Hy: 23
selection on frame 16
x: 244, y: 89, Hx: 10, Hy: 24
selection on frame 17
x: 243, y: 89, Hx: 10, Hy: 24
selection on frame 18
x: 242, y: 88, Hx: 9, Hy: 23
selection on frame 19
x: 242, y: 88, Hx: 9, Hy: 23
selection on frame 20
x: 2

In [26]:
pf_df.loc[len(pf_df)] = [name, pf_time, len(PF.particles)]
pf_df.to_csv('./output/pf_df.csv', index=False)

In [27]:
pf_df

,Test,Time,Particle Count
0,MOT17-09-DPM-raw,2087.300000,50
1,MOT17-09-DPM-raw,496.561201,10
2,MOT17-09-DPM-raw,3197.979189,80


In [28]:
save_output(output_name=f'{name}_{PF.particle_count}_{PF.SCALE_CHANGE_D}.mp4')

0